In [1]:
# This script examines the UCI sanctioned racing calendar to look at global changes in the number of events.
# Source is https://www.uci.org/road/calendar, all categories/classes selected
# Year could be extracted from the start date, but UCI 'seasons' differ from calendar years
# for Africa, Asia and Oceania circuits, so when outputting new calendars, add the year as _2XXX before .xlsx
# The script will add the year as a new column.
%matplotlib inline

In [2]:
import pandas as pd
import glob
import os
import time
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#Map the UCI Calendar excel output to the Continental circuits using the UCI's country names
continents = {'Americas': ['ANTIGUA AND BARBUDA', 'ARGENTINA', 'ARUBA', 'BELIZE', 'BERMUDA', 
                           'BOLIVARIAN REPUBLIC OF VENEZUELA', 'BOLIVIA', 'BRAZIL', 'CANADA',
                           'CHILE', 'COLOMBIA', 'CUBA', 'DOMINICAN REPUBLIC', 'ECUADOR', 
                           'EL SALVADOR', 'HONDURAS', 'PANAMA', 'PARAGUAY', 'PUERTO RICO', 
                           'SAINT VINCENT AND THE GRENADINES', 'TRINIDAD AND TOBAGO', 
                           'UNITED STATES OF AMERICA', 'URUGUAY','MEXICO', 'COSTA RICA','GUATEMALA'],
              'Africas' : ['ALBANIA', 'ALGERIA', 'ANGOLA', 'CAMEROON', 'CONGO', 'COTE D\'IVOIRE',
                           'EGYPT', 'ETHIOPIA', 'GUYANA', 'LIBYA', 'MALI', 'MAURITIUS', 'MOROCCO',
                           'NAMIBIA', 'RWANDA', 'SENEGAL', 'SOUTH AFRICA', 'SWAZILAND', 'TUNISIA', 
                           'UGANDA', 'ZIMBABWE', 'GABON','ERITREA', 'BURKINA FASO'],
              'Asia' : ['AZERBAIJAN', 'BAHRAIN', 'BRUNEI DARUSSALAM', 'CHINESE TAIPEI', 'GEORGIA',
                        'INDIA', 'ISLAMIC REPUBLIC OF IRAN', 'ISRAEL', 'KAZAKHSTAN', 'KOREA', 'KUWAIT', 
                        'KYRGYZSTAN', 'LEBANON', 'MALAYSIA', 'MONGOLIA', 'MYANMAR', 'OMAN','PAKISTAN', 'PHILIPPINES',
                        'RUSSIAN FEDERATION', 'SINGAPORE', 'SRI LANKA', 'SYRIAN ARAB REPUBLIC', 'THAILAND',
                        'TURKEY', 'UNITED ARAB EMIRATES', 'VIETNAM','HONG KONG, CHINA','QATAR', 'UZBEKISTAN',
                        'INDONESIA', 'JAPAN','PEOPLE\'S REPUBLIC OF CHINA'],
              'Europe' : ['AUSTRIA', 'BELARUS', 'BELGIUM', 'BOSNIA AND HERZEGOVINA', 'BULGARIA', 'CROATIA',
                          'CYPRUS', 'CZECH REPUBLIC', 'DENMARK', 'ESTONIA', 'FINLAND', 
                          'FORMER YUGOSLAV REPUBLIC OF MACEDONIA', 'FRANCE', 'GERMANY', 'GREAT BRITAIN', 
                          'GREECE', 'HUNGARY', 'ICELAND', 'IRELAND', 'ITALY', 'KOSOVO', 'LATVIA', 'LITHUANIA',
                          'LUXEMBOURG', 'MONTENEGRO', 'NETHERLANDS', 'NORWAY', 'POLAND', 'PORTUGAL', 
                          'REPUBLIC OF MOLDOVA', 'ROMANIA', 'SAN MARINO', 'SERBIA', 
                          'SLOVAKIA', 'SLOVENIA', 'SWEDEN', 'SWITZERLAND', 'UKRAINE','SPAIN'],
              'Oceania' : ['NEW ZEALAND','AUSTRALIA']}

In [3]:
#Pull in every excel sheet from the current folder that starts with Calendar and ends in xlsx. YMMV!

all_data = pd.DataFrame()
for f in glob.glob('./data/Calendar*.xlsx'):
    df = pd.read_excel(f, header=1, encoding='UTF-8')
    year = os.path.basename(f).split('.')[0].split('_')[-1]
    df['Season'] = year
    all_data = all_data.append(df,ignore_index=True)
    
len(all_data)

9542

In [4]:
#Get rid of unwanted columns.
all_data = all_data.drop(columns=['EMail', 'WebSite', 'Calendar','Venue'])
all_data.head()

,Date From,Date To,Name,Country,Category,Class,Season
0,24/10/2017,29/10/2017,Vuelta a Colombia Femenina Oro y Paz,COLOMBIA,WE,2.2,2018
1,23/10/2017,01/11/2017,Vuelta a Guatemala,GUATEMALA,ME,2.2,2018
2,04/11/2017,04/11/2017,Subaru Australian Open Criterium,AUSTRALIA,ME,CRT,2018
3,04/11/2017,04/11/2017,Le Tour De France Saitama Criterium,JAPAN,ME,CRT,2018
4,04/11/2017,04/11/2017,Subaru Australian Open Criterium,AUSTRALIA,WE,CRT,2018


In [5]:
#Examine the head and length to make sure it's kosher
# all_data['Class'].unique()
# array(['2.2', 'CRT', '2.HC', '2.1', '1.2', '1.1', 'CN', '2.UWT', '1.UWT',
#        '2.Ncup', '1.HC', 'CC', '1.WWT', '1.Ncup', '1.2U', '2.2U', 'JR',
#        '2.WWT', 'JC', 'CM', 'JOJ', 'CDM', nan, 'JO', 'CPE', 'MNM', 'AU1',
#        '2.CH', '1.CH', 'GT2', 'GT1'], dtype=object)
all_data['Category'] = all_data['Category'].replace('MP', 'ME')
all_data['Category'].unique()

array(['WE', 'ME', nan, 'MU', 'MJ', 'WJ', 'WU'], dtype=object)

In [6]:
# Championship races lack categories because they're usually combined all cats.
len(all_data.loc[all_data['Category'].isna()])

393

In [7]:
# The championships, Olympics, Criteriums (some) lack a category and can be eliminated assuming they're equal ME/WE
noc_data = all_data.loc[all_data['Category'].notnull()]
noc_data.loc[noc_data['Class'].isna()]
# There are still two races without Class, we can drop

,Date From,Date To,Name,Country,Category,Class,Season
2128,21/11/2018,25/11/2018,Africa Cup - Eritrea,ERITREA,WE,NaN,2019
2129,21/11/2018,25/11/2018,Africa Cup - Eritrea,ERITREA,ME,NaN,2019


In [9]:
noc_data = noc_data.loc[noc_data['Class'].notnull()]
noc_data.shape

(9147, 7)

In [10]:
# I suspect there are some classes have changed over the years. MP is ME, CPE was WT stage races during dispute
# with ASO (2.UWT), MNM = Monument and CPE were WT one-days during disputed years (1.UWT) 
# Reassign them to current codes. This may not be necessary in the future.
class_dict = {'CPE':'2.UWT',
              'MNM':'1.UWT',
              'AU1':'1.UWT',
              '2.CH':'2.UWT',
              '1.CH':'1.UWT',
              'GT2':'2.UWT',
              'GT1':'2.UWT',
              '2.2':'2.2','CRT':'CRT','2.HC':'2.HC','2.1':'2.1','1.2':'1.2','1.1':'1.1',
              'CN':'CN','2.UWT':'2.UWT','1.UWT':'1.UWT','2.Ncup':'2.Ncup','1.HC':'1.HC','CC':'CC',
              '1.WWT':'1.WWT','1.Ncup':'1.Ncup','1.2U':'1.2U','2.2U':'2.2U','JR':'JR','2.WWT':'2.WWT',
              'JC':'JC','CM':'CM','JOJ':'JOJ','CDM':'CDM','JO':'JO'}

noc_data['Class'] = noc_data['Class'].map(class_dict.get)

In [11]:
noc_data['Class'].unique()

array(['2.2', 'CRT', '2.HC', '2.1', '1.2', '1.1', '2.UWT', '1.UWT',
       '2.Ncup', '1.HC', '1.WWT', '1.Ncup', '1.2U', '2.2U', '2.WWT', 'CN',
       'CC', 'JR', 'CDM', 'JOJ', 'CM', 'JO'], dtype=object)

In [12]:
noc_data.loc[noc_data['Country'].isna()]

,Date From,Date To,Name,Country,Category,Class,Season
6229,27/04/2006,27/04/2006,Trofej Sajamskih Gradova,NaN,MJ,1.1,2006
6269,12/05/2006,12/05/2006,Trofej Sajamskih Gradova,NaN,MJ,1.1,2006


In [13]:
drop_class = ['JO','JOJ','CC','JR','CDM','CN','CRT']

keep_class = ['2.2',  '2.HC', '2.1', '1.2', '1.1', '2.UWT', '1.UWT', 
              '2.Ncup', '1.HC', '1.WWT', '1.Ncup', '1.2U', '2.2U', '2.WWT']

non_champs = noc_data.loc[noc_data['Class'].isin(keep_class)]
champs = all_data.loc[all_data['Class'].isin(drop_class)]
non_champs.shape

(7068, 7)

In [14]:
champs['Class'].unique()
# These are the races we won't consider further for now.

array(['CRT', 'CN', 'CC', 'JR', 'JOJ', 'CDM', 'JO'], dtype=object)

In [15]:
#Need to rename the columns so they make sense and format the dates as dates
non_champs = non_champs.rename(columns={'Date From':'Start_date', 'Date To':'End_Date'})
non_champs['Start_date'] = pd.to_datetime(non_champs['Start_date'], dayfirst=True)
non_champs['End_Date'] = pd.to_datetime(non_champs['End_Date'], dayfirst=True)
non_champs.head()

,Start_date,End_Date,Name,Country,Category,Class,Season
0,2017-10-24,2017-10-29,Vuelta a Colombia Femenina Oro y Paz,COLOMBIA,WE,2.2,2018
1,2017-10-23,2017-11-01,Vuelta a Guatemala,GUATEMALA,ME,2.2,2018
5,2017-10-27,2017-11-05,Tour du Faso,BURKINA FASO,ME,2.2,2018
6,2017-10-27,2017-11-05,Vuelta Ciclista a Venezuela,BOLIVARIAN REPUBLIC OF VENEZUELA,ME,2.2,2018
7,2017-10-28,2017-11-05,Tour of Hainan,PEOPLE'S REPUBLIC OF CHINA,ME,2.HC,2018


In [16]:
#Adding in a column for the length of the race in days
non_champs['Race_Days'] = ((non_champs['End_Date'] + pd.DateOffset(days=1)) - non_champs['Start_date'])
non_champs.head()


,Start_date,End_Date,Name,Country,Category,Class,Season,Race_Days
0,2017-10-24,2017-10-29,Vuelta a Colombia Femenina Oro y Paz,COLOMBIA,WE,2.2,2018,6 days
1,2017-10-23,2017-11-01,Vuelta a Guatemala,GUATEMALA,ME,2.2,2018,10 days
5,2017-10-27,2017-11-05,Tour du Faso,BURKINA FASO,ME,2.2,2018,10 days
6,2017-10-27,2017-11-05,Vuelta Ciclista a Venezuela,BOLIVARIAN REPUBLIC OF VENEZUELA,ME,2.2,2018,10 days
7,2017-10-28,2017-11-05,Tour of Hainan,PEOPLE'S REPUBLIC OF CHINA,ME,2.HC,2018,9 days


In [17]:
# ENECO TOUR is 'stateless' because it crosses boundaries. We can add it to belgium.
non_champs['Country'].replace({'STATELESS':'BELGIUM'}, inplace=True)
non_champs['Country'].unique()

array(['COLOMBIA', 'GUATEMALA', 'BURKINA FASO',
       'BOLIVARIAN REPUBLIC OF VENEZUELA', "PEOPLE'S REPUBLIC OF CHINA",
       'JAPAN', 'RWANDA', 'SOUTH AFRICA', 'INDONESIA', 'COSTA RICA',
       'AUSTRALIA', 'GABON', 'NEW ZEALAND', 'SPAIN',
       'UNITED ARAB EMIRATES', 'ARGENTINA', 'FRANCE', 'ALGERIA',
       'CAMEROON', 'TURKEY', 'ITALY', 'OMAN', 'PORTUGAL', 'CROATIA',
       'BELGIUM', 'SLOVENIA', 'GREECE', 'NETHERLANDS', 'CHINESE TAIPEI',
       'TUNISIA', 'MALAYSIA', 'SLOVAKIA', 'URUGUAY', 'LUXEMBOURG',
       'THAILAND', 'GERMANY', 'MOROCCO', 'UNITED STATES OF AMERICA',
       'CZECH REPUBLIC', 'POLAND', 'GREAT BRITAIN', 'DENMARK', 'SENEGAL',
       'SWITZERLAND', 'BOSNIA AND HERZEGOVINA', 'NORWAY',
       'RUSSIAN FEDERATION', 'SRI LANKA', 'BULGARIA', 'SWEDEN', 'HUNGARY',
       'ALBANIA', 'PHILIPPINES', 'ESTONIA', 'IRELAND', 'UKRAINE', 'KOREA',
       'ROMANIA', 'CANADA', 'SERBIA', 'AUSTRIA', 'BELARUS',
       "COTE D'IVOIRE", 'KAZAKHSTAN', 'ISLAMIC REPUBLIC OF IRAN',
      

In [18]:
#add in the coordinates for each race so we can geolocate them on a map later

uci_country_coord = pd.DataFrame()
uci_country_coord = pd.read_excel('./data/uci_country_coord.xlsx', encoding='UTF-8')
uci_df = pd.merge(non_champs, uci_country_coord, right_on='uci_name', left_on='Country', how="left")
#Find which ones don't have coordinates...
df_nulls = uci_df[uci_df.isnull().any(axis=1)]
# loc_nulls = df_nulls.groupby('Country')
# loc_nulls['Country'].value_counts()
df_nulls

,Start_date,End_Date,Name,Country,Category,Class,Season,Race_Days,uci_name,continent,name,country,latitude,longitude
1666,2019-04-13,2019-04-13,Elite Road Central American Championships - ME...,NICARAGUA,ME,1.2,2019,1 days,NaN,NaN,NaN,NaN,NaN,NaN
1667,2019-04-13,2019-04-13,Elite Road Central American Championships - WE...,NICARAGUA,WE,1.2,2019,1 days,NaN,NaN,NaN,NaN,NaN,NaN
1678,2019-04-14,2019-04-14,Elite Road Central American Championships - ME...,NICARAGUA,ME,1.2,2019,1 days,NaN,NaN,NaN,NaN,NaN,NaN
1679,2019-04-14,2019-04-14,Elite Road Central American Championships - WE...,NICARAGUA,WE,1.2,2019,1 days,NaN,NaN,NaN,NaN,NaN,NaN
4708,2006-04-27,2006-04-27,Trofej Sajamskih Gradova,NaN,MJ,1.1,2006,1 days,NaN,NaN,NaN,NaN,NaN,NaN
4747,2006-05-12,2006-05-12,Trofej Sajamskih Gradova,NaN,MJ,1.1,2006,1 days,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
uci_df.shape


(7068, 14)

In [203]:
#You can extract the year from the start date, but UCI 'seasons' differ from calendar years
# for Africa, Asia and Oceania circuits, so when outputting new calendars, add the year as _2XXX before .xlsx
# If you want you can just export the combined data

# all_data['Year'] = pd.DatetimeIndex(all_data['Date From']).year

# writer = pd.ExcelWriter('combined.xlsx')
# all_data.to_excel(writer,'Sheet1')

# writer.save()

In [20]:
#We need to make the dictionary of lists one big dictionary to add the Continents column

cont_dict_converted = {k: oldk for oldk, oldv in continents.items() for k in oldv}

#Note there are some events that cross borders or Continental championships that have no Continent.
#Add some code here to handle these cases.

uci_df['Continent'] = uci_df['Country'].map(cont_dict_converted).fillna('Stateless')


In [21]:
# Get rid of the one Junior race that was in Serbia but lacked a country
uci_df = uci_df.loc[uci_df['Continent']!='Stateless']

In [206]:
# #Let's focus for now on the elite men and women's caregories in non-championship races

# cats = ['ME', 'WE']
# non_championships_df.Category.isin(cats)
# elite_non_championships = non_championships_df[non_championships_df.Category.isin(cats)]
# # group_cont = elite_non_championships.groupby("Continent")
# # group_cont.count().head()

# elite_non_championships.head()

In [22]:
#Convert the race days column to integer
uci_df['Race_Days'] = uci_df['Race_Days']/np.timedelta64(1, 'D')
uci_df.head()
len(uci_df)

7062

In [26]:
uci_df.to_csv('./data/cleaned_uci_data_12_19.csv', header=True, encoding='UTF-8')

In [25]:
uci_df.groupby(['Season','Category']).count()

Start_date  End_Date  Name  Country  Class  Race_Days  \
Season Category                                                          
2006   ME               375       375   375      375    375        375   
       MJ                51        51    51       51     51         51   
       MU                15        15    15       15     15         15   
       WE                43        43    43       43     43         43   
       WJ                 1         1     1        1      1          1   
2007   ME               388       388   388      388    388        388   
       MJ                51        51    51       51     51         51   
       MU                22        22    22       22     22         22   
       WE                49        49    49       49     49         49   
       WJ                 2         2     2        2      2          2   
2008   ME               396       396   396      396    396        396   
       MJ                55        55    55       55     55         55   
       MU                23        23    23       23     23         23   
       WE                61        61    61       61     61         61   
       WJ                 4         4     4        4      4          4   
2009   ME               367       367   367      367    367        367   
       MJ                55        55    55       55     55         55   
       MU                21        21    21       21     21         21   
       WE                50        50    50       50     50         50   
       WJ                 2         2     2        2      2          2   
2010   ME               364       364   364      364    364        364   
       MJ                52        52    52       52     52         52   
       MU                21        21    21       21     21         21   
       WE                42        42    42       42     42         42   
       WJ                 2         2     2        2      2          2   
2011   ME               375       375   375      375    375        375   
       MJ                44        44    44       44     44         44   
       MU                21        21    21       21     21         21   
       WE                52        52    52       52     52         52   
       WJ                 2         2     2        2      2          2   
...                     ...       ...   ...      ...    ...        ...   
2014   ME               361       361   361      361    361        361   
       MJ                36        36    36       36     36         36   
       MU                22        22    22       22     22         22   
       WE                65        65    65       65     65         65   
       WJ                 1         1     1        1      1          1   
2015   ME               381       381   381      381    381        381   
       MJ                36        36    36       36     36         36   
       MU                22        22    22       22     22         22   
       WE                73        73    73       73     73         73   
       WJ                 2         2     2        2      2          2   
2016   ME               340       340   340      340    340        340   
       MJ                42        42    42       42     42         42   
       MU                26        26    26       26     26         26   
       WE                92        92    92       92     92         92   
       WJ                 5         5     5        5      5          5   
2017   ME               363       363   363      363    363        363   
       MJ                47        47    47       47     47         47   
       MU                28        28    28       28     28         28   
       WE                75        75    75       75     75         75   
       WJ                 5         5     5        5      5          5   
2018   ME               353       353   353      353    353        353   
       MJ            